In [122]:
import pandas as pd
import json
list_of_dicts = [{'destination_addresses': ['95 Woodstown Rd, Swedesboro, NJ 08085, USA'], 'origin_addresses': ['211 S Christopher Columbus Blvd, Philadelphia, PA 19106, USA'], 'rows': [{'elements': [{'distance': {'text': '28.9 mi', 'value': 46501}, 'duration': {'text': '37 mins', 'value': 2248}, 'duration_in_traffic': {'text': '38 mins', 'value': 2300}, 'status': 'OK'}]}], 'status': 'OK'}, {'destination_addresses': ['164 S Broad St, Woodbury, NJ 08096, USA'], 'origin_addresses': ['211 S Christopher Columbus Blvd, Philadelphia, PA 19106, USA'], 'rows': [{'elements': [{'distance': {'text': '13.6 mi', 'value': 21861}, 'duration': {'text': '23 mins', 'value': 1354}, 'duration_in_traffic': {'text': '24 mins', 'value': 1449}, 'status': 'OK'}]}], 'status': 'OK'}, {'destination_addresses': ['5 W Mantua Ave, Wenonah, NJ 08090, USA'], 'origin_addresses': ['211 S Christopher Columbus Blvd, Philadelphia, PA 19106, USA'], 'rows': [{'elements': [{'distance': {'text': '16.9 mi', 'value': 27214}, 'duration': {'text': '25 mins', 'value': 1470}, 'duration_in_traffic': {'text': '27 mins', 'value': 1599}, 'status': 'OK'}]}], 'status': 'OK'}, {'destination_addresses': ['480 Mullica Hill Rd, Mullica Hill, NJ 08062, USA'], 'origin_addresses': ['211 S Christopher Columbus Blvd, Philadelphia, PA 19106, USA'], 'rows': [{'elements': [{'distance': {'text': '21.4 mi', 'value': 34494}, 'duration': {'text': '27 mins', 'value': 1603}, 'duration_in_traffic': {'text': '29 mins', 'value': 1720}, 'status': 'OK'}]}], 'status': 'OK'}]

In [123]:
def unpack_dicts(list_of_dicts):
    """function to crack into the nested dictionary structure that the api returns"""
    df = pd.DataFrame()
    df['Address'] = ""
    current_index = 0
    for dict in list_of_dicts:
        destinations = pd.Series(json.dumps(dict["destination_addresses"][0]))
        json_object = json.dumps(dict["rows"][0].get("elements")[0])
        df_dict = pd.read_json(json_object)
        df_dict = df_dict.drop(labels=["value"], axis=0)
        df = pd.concat([df, df_dict], ignore_index=True, axis = 0)
        df.at[current_index, "Address"] = destinations[0]
        current_index += 1

    return df

unpack_dicts(list_of_dicts)


,Address,distance,duration,duration_in_traffic,status
0,"""95 Woodstown Rd, Swedesboro, NJ 08085, USA""",28.9 mi,37 mins,38 mins,OK
1,"""164 S Broad St, Woodbury, NJ 08096, USA""",13.6 mi,23 mins,24 mins,OK
2,"""5 W Mantua Ave, Wenonah, NJ 08090, USA""",16.9 mi,25 mins,27 mins,OK
3,"""480 Mullica Hill Rd, Mullica Hill, NJ 08062, ...",21.4 mi,27 mins,29 mins,OK
